In [ ]:
!pip install pytrends circlify pandas matplotlib seaborn openpyxl

In [ ]:
import pandas as pd
import time
from pytrends.request import TrendReq
from random import randint

# --- KONFIGURASI ROSTER TIM ---
official_rosters = {
    "Team Love": ["Alya", "Anindya", "Nayla", "Lia", "Lana", "Cathy", "Elin", "Cynthia", "Fiony", "Fritzy", "Gracie", "Lily", "Indah", "Trisha", "Michie"],
    "Team Dream": ["Delynn", "Amanda", "Chelsea", "Olla", "Freya", "Ella", "Gendis", "Gita", "Greesel", "Eli", "Lyn", "Marsha", "Nachia", "Oline", "Nala"],
    "Team Passion": ["Aralie", "Christy", "Erine", "Oniel", "Danella", "Daisy", "Feni", "Jessi", "Kathrina", "Lulu", "Levi", "Muthe", "Raisha", "Ribka", "Kimmy"]
}

# Setup Engine
pytrends = TrendReq(hl='id-ID', tz=420)
TIMEFRAME = '2026-01-01 2026-01-31'
ANCHOR_KEYWORD = "JKT48"
full_database = {}

print(f"🚀 MEMULAI MINING JKT48 NEW ERA...")
print(f"   Target: Memisahkan Top vs Rising Queries (Raw Data).")

# Flatten roster
all_targets = []
for team, members in official_rosters.items():
    for m in members:
        all_targets.append({'name': m, 'query': f"{m} JKT48", 'team': team})

# --- LOOPING DATA ---
for index, target in enumerate(all_targets, 1):
    name_key = target['name']
    query = target['query']
    team = target['team']

    success = False
    attempts = 0

    while not success:
        try:
            print(f"[{index}/{len(all_targets)}] {name_key}...", end=" ")

            # Request Utama
            pytrends.build_payload([ANCHOR_KEYWORD, query], cat=0, timeframe=TIMEFRAME, geo='ID')

            # 1. Score & Trend
            iot = pytrends.interest_over_time()
            if not iot.empty:
                anchor_val = iot[ANCHOR_KEYWORD].mean() or 1
                raw_score = iot[query].mean()
                norm_score = (raw_score / anchor_val) * 100
                daily_trend = iot[query].values.tolist()
            else:
                norm_score = 0; daily_trend = [0]*31

            time.sleep(2) # Napas

            # 2. Wilayah
            ibr = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True)
            top_regions = ibr[query].sort_values(ascending=False).head(5).to_dict() if not ibr.empty else {}

            # 3. Related Queries (DIPISAH & RAW)
            rq = pytrends.related_queries()
            kw_top = []
            kw_rising = []

            if rq and query in rq:
                # Ambil Top 5
                if rq[query]['top'] is not None:
                    kw_top = rq[query]['top']['query'].head(5).tolist()
                # Ambil Rising 5
                if rq[query]['rising'] is not None:
                    kw_rising = rq[query]['rising']['query'].head(5).tolist()

            # Handling Empty
            if not kw_top: kw_top = ["-"]
            if not kw_rising: kw_rising = ["-"]

            # Simpan
            full_database[name_key] = {
                'full_name': query, 'team': team,
                'score': norm_score, 'daily_trend': daily_trend,
                'top_regions': top_regions,
                'kw_top': kw_top,       # List Terpisah
                'kw_rising': kw_rising  # List Terpisah
            }

            print(f"✅ OK. (T:{len(kw_top)} R:{len(kw_rising)})")
            success = True
            time.sleep(randint(10, 15)) # Jeda Wajib

        except Exception as e:
            attempts += 1
            wait = 45 * attempts
            print(f"\n   ⚠️ Limit Google (Error 429). Tunggu {wait} detik...")
            time.sleep(wait)
            pytrends = TrendReq(hl='id-ID', tz=420)

print("\n🎉 MINING SELESAI! Data sudah di memori.")

🚀 MEMULAI MINING JKT48 NEW ERA...
   Target: Memisahkan Top vs Rising Queries (Raw Data).
[1/45] Alya... ✅ OK. (T:1 R:1)
[2/45] Anindya... ✅ OK. (T:1 R:1)
[3/45] Nayla... ✅ OK. (T:1 R:1)
[4/45] Lia... ✅ OK. (T:1 R:1)
[5/45] Lana... ✅ OK. (T:1 R:1)
[6/45] Cathy... ✅ OK. (T:1 R:1)
[7/45] Elin... ✅ OK. (T:1 R:1)
[8/45] Cynthia... ✅ OK. (T:1 R:1)
[9/45] Fiony... ✅ OK. (T:1 R:1)
[10/45] Fritzy... ✅ OK. (T:1 R:1)
[11/45] Gracie... ✅ OK. (T:1 R:1)
[12/45] Lily... ✅ OK. (T:1 R:1)
[13/45] Indah... ✅ OK. (T:1 R:1)
[14/45] Trisha... ✅ OK. (T:1 R:1)
[15/45] Michie... ✅ OK. (T:2 R:2)
[16/45] Delynn... ✅ OK. (T:1 R:1)
[17/45] Amanda... ✅ OK. (T:1 R:1)
[18/45] Chelsea... ✅ OK. (T:1 R:1)
[19/45] Olla... ✅ OK. (T:1 R:1)
[20/45] Freya... ✅ OK. (T:5 R:5)
[21/45] Ella... ✅ OK. (T:1 R:1)
[22/45] Gendis... ✅ OK. (T:1 R:1)
[23/45] Gita... ✅ OK. (T:1 R:1)
[24/45] Greesel... ✅ OK. (T:1 R:1)
[25/45] Eli... ✅ OK. (T:1 R:1)
[26/45] Lyn... ✅ OK. (T:1 R:1)
[27/45] Marsha... ✅ OK. (T:5 R:5)
[28/45] Nachia... ✅ OK. (

In [ ]:
import pandas as pd

nama_file = "Laporan_JKT48_NewEra_Lengkap_45Member.xlsx"
print(f"💾 Sedang menyimpan database ke Excel: {nama_file}...")

excel_rows = []
for name, info in full_database.items():
    # Format Wilayah
    regions_str = ", ".join([f"{k} ({v})" for k, v in info['top_regions'].items()])

    # Format Keywords (Top & Rising dipisah " | ")
    top_str = " | ".join(info['kw_top'])
    rising_str = " | ".join(info['kw_rising'])

    excel_rows.append({
        'Nama Member': name,
        'Tim': info['team'],
        'Score (0-100)': info['score'],
        'Top Queries (Volume)': top_str,
        'Rising Queries (Breakout)': rising_str,
        'Top Wilayah': regions_str
    })

# Urutkan berdasarkan Score tertinggi
df_excel = pd.DataFrame(excel_rows).sort_values(by='Score (0-100)', ascending=False)
df_excel.to_excel(nama_file, index=False)

print(f"✅ FILE EXCEL AMAN! Silakan download '{nama_file}' di folder.")

💾 Sedang menyimpan database ke Excel: Laporan_JKT48_NewEra_Lengkap_45Member.xlsx...
✅ FILE EXCEL AMAN! Silakan download 'Laporan_JKT48_NewEra_Lengkap_45Member.xlsx' di folder.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import circlify
import pandas as pd
import numpy as np
import os

# --- SETUP OUTPUT ---
output_folder = "JKT48_Posters_Final_45Member"
if not os.path.exists(output_folder): os.makedirs(output_folder)

# Gradasi Kuning (High) ke Hijau (Low)
cmap_name = 'GreenToYellow'
cm = mcolors.LinearSegmentedColormap.from_list(cmap_name, ["#00E676", "#FFD600"], N=100)

# Warna Identitas Tim
team_colors = {
    "Team Love": "#FF69B4",    # Hot Pink
    "Team Dream": "#00BFFF",   # Deep Sky Blue
    "Team Passion": "#FF4500"  # Orange Red
}

print(f"🎨 MEMULAI GENERASI POSTER (45 Member)...")

# ==========================================
# 1. BUBBLE CHART (OVERVIEW TIM)
# ==========================================
print("   [1/2] Mencetak Bubble Chart Utama...")
data_list = [{'Member': k, 'Score': v['score'], 'Team': v['team']} for k, v in full_database.items()]
df_bubble = pd.DataFrame(data_list).sort_values(by='Score', ascending=False).head(25) # Top 25 biar ga penuh
df_bubble['Score_Vis'] = df_bubble['Score'].apply(lambda x: 0.1 if x <= 0 else x)

circles = circlify.circlify(df_bubble['Score_Vis'].tolist(), show_enclosure=False, target_enclosure=circlify.Circle(x=0, y=0, r=1))
circles = circles[::-1]

fig, ax = plt.subplots(figsize=(12, 12))
fig.patch.set_facecolor('#121212'); ax.set_facecolor('#121212')
ax.axis('off'); ax.set_xlim(-1, 1); ax.set_ylim(-1, 1)

plt.text(0, 1.08, "JKT48 SEARCH UNIVERSE", ha='center', fontsize=30, weight='bold', color='#FFD600')
plt.text(0, 1.03, "Overview Popularitas Tim | Jan 2026", ha='center', fontsize=12, color='white')

max_score = df_bubble['Score'].max()
for circle, (idx, row) in zip(circles, df_bubble.iterrows()):
    x, y, r = circle.x, circle.y, circle.r
    score, team = row['Score'], row['Team']
    t_color = team_colors.get(team, '#FFF')
    f_color = cm(score / max_score if max_score > 0 else 0.5)

    ax.add_patch(plt.Circle((x, y), r, alpha=0.3, facecolor=f_color, edgecolor=t_color, linewidth=3))
    if r > 0.04:
        fs = r * 4.5
        ax.text(x, y+r*0.25, row['Member'].upper(), ha='center', va='center', fontsize=14*fs, weight='bold', color='white')
        ax.text(x, y-r*0.15, f"{score:.2f}", ha='center', va='center', fontsize=18*fs, weight='bold', color=t_color)
        ax.text(x, y-r*0.45, team.replace("Team ","").upper(), ha='center', va='center', fontsize=8*fs, color='#AAA')

plt.savefig(f"{output_folder}/00_Overview_Bubble.png", dpi=300, bbox_inches='tight', facecolor='#121212')
plt.close()

# ==========================================
# 2. POSTER INDIVIDUAL (DEEP DIVE)
# ==========================================
print("   [2/2] Mencetak Poster Satuan...")
sorted_members = sorted(full_database.items(), key=lambda x: x[1]['score'], reverse=True)

for rank, (name, info) in enumerate(sorted_members, 1):
    score, team = info['score'], info['team']
    daily = np.array(info['daily_trend'])
    t_color = team_colors.get(team, '#FFF')

    fig = plt.figure(figsize=(10, 12))
    fig.patch.set_facecolor('#121212')
    gs = fig.add_gridspec(3, 2, height_ratios=[1, 1.5, 1])

    # A. HEADER
    ax_h = fig.add_subplot(gs[0, :]); ax_h.axis('off'); ax_h.set_facecolor('#121212')
    ax_h.text(0.5, 0.7, name.upper(), ha='center', fontsize=50, weight='bold', color='#FFD600')
    ax_h.text(0.5, 0.55, f"  {team.upper()}  ", ha='center', fontsize=15, weight='bold', color='#121212',
              bbox=dict(facecolor=t_color, edgecolor='none', boxstyle='round,pad=0.4'))
    ax_h.text(0.5, 0.40, f"RANK #{rank}  |  INDEX: {score:.2f}", ha='center', fontsize=14, color='#AAA')

    # Gradient Line
    gl = np.linspace(0, 1, 100).reshape(1, -1)
    ax_l = fig.add_axes([0.15, 0.72, 0.7, 0.005]); ax_l.imshow(gl, aspect='auto', cmap=cm); ax_l.axis('off')

    # B. GRAFIK (Trend)
    ax_t = fig.add_subplot(gs[1, :]); ax_t.set_facecolor('#121212')
    days = np.arange(1, len(daily)+1)
    segs = np.concatenate([np.array([days, daily]).T.reshape(-1, 1, 2)[:-1], np.array([days, daily]).T.reshape(-1, 1, 2)[1:]], axis=1)
    lc = LineCollection(segs, cmap=cm, norm=plt.Normalize(daily.min(), daily.max()))
    lc.set_array(daily); lc.set_linewidth(3); ax_t.add_collection(lc); ax_t.autoscale_view()
    ax_t.fill_between(days, daily, color='#00E676', alpha=0.1)
    ax_t.set_title("📈 DAILY TREND", fontsize=12, color='white', loc='left')
    ax_t.tick_params(colors='white'); ax_t.axis('off')

    # C. REGIONS
    ax_r = fig.add_subplot(gs[2, 0]); ax_r.set_facecolor('#121212')
    provs = [p.replace("Daerah Istimewa","DI").replace("Kepulauan","Kep.") for p in info['top_regions'].keys()]
    vals = list(info['top_regions'].values())
    ax_r.barh(range(len(provs)), vals, color=t_color, alpha=0.8)
    ax_r.set_yticks(range(len(provs))); ax_r.set_yticklabels(provs, color='white', fontsize=9)
    ax_r.invert_yaxis(); ax_r.axis('off'); ax_r.set_title("🗺️ TOP WILAYAH", fontsize=11, color='white', loc='left')
    for i, v in enumerate(vals): ax_r.text(v+1, i, str(v), va='center', fontsize=9, color=t_color, weight='bold')

    # D. KEYWORDS (TOP & RISING)
    ax_k = fig.add_subplot(gs[2, 1]); ax_k.axis('off'); ax_k.set_facecolor('#121212')
    ax_k.set_title("🔍 KEYWORDS", fontsize=11, color='white', loc='left')
    y = 0.9

    # Logika Tampilan: Max 3 Top, Max 3 Rising
    kw_to_show = []
    if info['kw_top'] != ["-"]: kw_to_show.append(("[TOP]", info['kw_top'][:3]))
    if info['kw_rising'] != ["-"]: kw_to_show.append(("[RISING]", info['kw_rising'][:3]))

    if not kw_to_show:
        ax_k.text(0, y, "(Belum ada data spesifik)", fontsize=10, color='#666')

    for label, kws in kw_to_show:
        ax_k.text(0, y, label, fontsize=10, weight='bold', color='#FFD600'); y -= 0.1
        for k in kws:
            ax_k.text(0, y, f"• {k[:25]}", fontsize=9, color='white'); y -= 0.1
        y -= 0.05

    plt.tight_layout()
    plt.savefig(f"{output_folder}/Rank_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
    plt.close()

    if rank % 5 == 0: print(f"   ... {rank} poster selesai.")

print(f"\n✅ SELESAI SEMUA! Cek folder '{output_folder}'.")

🎨 MEMULAI GENERASI POSTER (45 Member)...
   [1/2] Mencetak Bubble Chart Utama...
   [2/2] Mencetak Poster Satuan...


/tmp/ipython-input-538418573.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Rank_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
/tmp/ipython-input-538418573.py:127: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Rank_{rank:02d}_{name}.png"

   ... 5 poster selesai.


/tmp/ipython-input-538418573.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-538418573.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Rank_{rank:02d}_{name}.png", dpi=300, facecolor='

   ... 10 poster selesai.


...

   ... 35 poster selesai.


/tmp/ipython-input-538418573.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-538418573.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Rank_{rank:02d}_{name}.png", dpi=300, facecolor='

   ... 40 poster selesai.


/tmp/ipython-input-538418573.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-538418573.py:126: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-538418573.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Rank_{rank:02d}_{name}.png", dpi=300, facecolor='

   ... 45 poster selesai.

✅ SELESAI SEMUA! Cek folder 'JKT48_Posters_Final_45Member'.


In [ ]:
!zip -r JKT48_Posters_Complete.zip JKT48_Posters_Final_45Member

  adding: JKT48_Posters_Final_45Member/ (stored 0%)
  adding: JKT48_Posters_Final_45Member/Rank_01_Freya.png (deflated 21%)
  adding: JKT48_Posters_Final_45Member/Rank_21_Fritzy.png (deflated 47%)
  adding: JKT48_Posters_Final_45Member/Rank_41_Kathrina.png (deflated 43%)
  adding: JKT48_Posters_Final_45Member/Rank_10_Indah.png (deflated 40%)
  adding: JKT48_Posters_Final_45Member/Rank_20_Cynthia.png (deflated 45%)
  adding: JKT48_Posters_Final_45Member/Rank_25_Delynn.png (deflated 46%)
  adding: JKT48_Posters_Final_45Member/Rank_32_Lyn.png (deflated 46%)
  adding: JKT48_Posters_Final_45Member/Rank_08_Fiony.png (deflated 39%)
  adding: JKT48_Posters_Final_45Member/Rank_40_Jessi.png (deflated 42%)
  adding: JKT48_Posters_Final_45Member/Rank_09_Levi.png (deflated 40%)
  adding: JKT48_Posters_Final_45Member/Rank_14_Anindya.png (deflated 44%)
  adding: JKT48_Posters_Final_45Member/Rank_11_Gita.png (deflated 36%)
  adding: JKT48_Posters_Final_45Member/Rank_23_Lily.png (deflated 50%)
  adding

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import circlify
import pandas as pd
import numpy as np
import os

# --- SETUP OUTPUT ---
output_folder = "JKT48_Posters_REVISI_Region"
if not os.path.exists(output_folder): os.makedirs(output_folder)

# Setup Warna
cmap_name = 'GreenToYellow'
cm = mcolors.LinearSegmentedColormap.from_list(cmap_name, ["#00E676", "#FFD600"], N=100)

team_colors = {
    "Team Love": "#FF69B4",    # Hot Pink
    "Team Dream": "#00BFFF",   # Deep Sky Blue
    "Team Passion": "#FF4500"  # Orange Red
}

print(f"🎨 MEMULAI REVISI POSTER (Memunculkan Nama Wilayah)...")

# Cek Data
if 'full_database' not in locals() or not full_database:
    print("⚠️ Data kosong! Pastikan Blok Mining sudah dijalankan.")
else:
    sorted_members = sorted(full_database.items(), key=lambda x: x[1]['score'], reverse=True)

    for rank, (name, info) in enumerate(sorted_members, 1):
        score, team = info['score'], info['team']
        daily = np.array(info['daily_trend'])
        t_color = team_colors.get(team, '#FFF')

        # Setup Canvas
        fig = plt.figure(figsize=(10, 12))
        fig.patch.set_facecolor('#121212')
        gs = fig.add_gridspec(3, 2, height_ratios=[1, 1.5, 1])

        # --- A. HEADER ---
        ax_h = fig.add_subplot(gs[0, :]); ax_h.axis('off'); ax_h.set_facecolor('#121212')
        ax_h.text(0.5, 0.7, name.upper(), ha='center', fontsize=50, weight='bold', color='#FFD600')
        ax_h.text(0.5, 0.55, f"  {team.upper()}  ", ha='center', fontsize=15, weight='bold', color='#121212',
                  bbox=dict(facecolor=t_color, edgecolor='none', boxstyle='round,pad=0.4'))
        ax_h.text(0.5, 0.40, f"RANK #{rank}  |  INDEX: {score:.2f}", ha='center', fontsize=14, color='#AAA')

        # Garis Gradasi
        gl = np.linspace(0, 1, 100).reshape(1, -1)
        ax_l = fig.add_axes([0.15, 0.72, 0.7, 0.005]); ax_l.imshow(gl, aspect='auto', cmap=cm); ax_l.axis('off')

        # --- B. GRAFIK (Trend) ---
        ax_t = fig.add_subplot(gs[1, :]); ax_t.set_facecolor('#121212')
        days = np.arange(1, len(daily)+1)
        segs = np.concatenate([np.array([days, daily]).T.reshape(-1, 1, 2)[:-1], np.array([days, daily]).T.reshape(-1, 1, 2)[1:]], axis=1)
        lc = LineCollection(segs, cmap=cm, norm=plt.Normalize(daily.min(), daily.max()))
        lc.set_array(daily); lc.set_linewidth(3); ax_t.add_collection(lc); ax_t.autoscale_view()
        ax_t.fill_between(days, daily, color='#00E676', alpha=0.1)
        ax_t.set_title("📈 DAILY TREND", fontsize=12, color='white', loc='left')
        ax_t.tick_params(colors='white'); ax_t.axis('off')

        # --- C. REGIONS (PERBAIKAN DI SINI) ---
        ax_r = fig.add_subplot(gs[2, 0]); ax_r.set_facecolor('#121212')

        provs = [p.replace("Daerah Istimewa","DI").replace("Kepulauan","Kep.").replace("Sumatera","Sumatra") for p in info['top_regions'].keys()]
        vals = list(info['top_regions'].values())

        # Gambar Bar
        ax_r.barh(range(len(provs)), vals, color=t_color, alpha=0.8)

        # SETTING SENSITIF: Jangan pakai axis('off'), tapi matikan spine satu-satu
        ax_r.spines['top'].set_visible(False)
        ax_r.spines['right'].set_visible(False)
        ax_r.spines['bottom'].set_visible(False)
        ax_r.spines['left'].set_visible(False) # Hilangkan garis vertikal kiri
        ax_r.get_xaxis().set_visible(False)    # Hilangkan angka bawah

        # Munculkan Nama Wilayah
        ax_r.set_yticks(range(len(provs)))
        ax_r.set_yticklabels(provs, color='white', fontsize=10, weight='bold') # Font diperbesar dikit
        ax_r.tick_params(axis='y', length=0) # Hilangkan strip kecil di samping teks

        ax_r.invert_yaxis()
        ax_r.set_title("🗺️ TOP WILAYAH", fontsize=11, color='white', loc='left', pad=10)

        # Angka Skor di ujung bar
        for i, v in enumerate(vals):
            ax_r.text(v+1, i, str(v), va='center', fontsize=9, color=t_color, weight='bold')

        # --- D. KEYWORDS ---
        ax_k = fig.add_subplot(gs[2, 1]); ax_k.axis('off'); ax_k.set_facecolor('#121212')
        ax_k.set_title("🔍 KEYWORDS", fontsize=11, color='white', loc='left')
        y = 0.9

        kw_to_show = []
        if info['kw_top'] != ["-"]: kw_to_show.append(("[TOP]", info['kw_top'][:3]))
        if info['kw_rising'] != ["-"]: kw_to_show.append(("[RISING]", info['kw_rising'][:3]))

        if not kw_to_show:
            ax_k.text(0, y, "(Belum ada data spesifik)", fontsize=10, color='#666')

        for label, kws in kw_to_show:
            ax_k.text(0, y, label, fontsize=10, weight='bold', color='#FFD600'); y -= 0.1
            for k in kws:
                # Karena skor tidak ada, kita pakai bullet point
                ax_k.text(0, y, f"• {k[:25]}", fontsize=9, color='white'); y -= 0.1
            y -= 0.05

        plt.tight_layout()
        plt.savefig(f"{output_folder}/Revisi_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
        plt.close()

        if rank % 5 == 0: print(f"   ... {rank} poster selesai.")

    print(f"\n✅ REVISI SELESAI! Nama Wilayah sudah muncul. Cek folder '{output_folder}'.")

🎨 MEMULAI REVISI POSTER (Memunculkan Nama Wilayah)...


/tmp/ipython-input-1625205180.py:110: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:111: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Revisi_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
/tmp/ipython-input-1625205180.py:111: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Revisi_{rank:02d}_{

   ... 5 poster selesai.


/tmp/ipython-input-1625205180.py:110: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:111: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Revisi_{rank:02d}_{name}.png", dpi=300, fac

   ... 10 poster selesai.


...

   ... 35 poster selesai.


/tmp/ipython-input-1625205180.py:110: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:111: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Revisi_{rank:02d}_{name}.png", dpi=300, fac

   ... 40 poster selesai.


/tmp/ipython-input-1625205180.py:110: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:110: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-1625205180.py:111: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Revisi_{rank:02d}_{name}.png", dpi=300, fac

   ... 45 poster selesai.

✅ REVISI SELESAI! Nama Wilayah sudah muncul. Cek folder 'JKT48_Posters_REVISI_Region'.


In [ ]:
!zip -r JKT48_Revisi_Region.zip JKT48_Posters_REVISI_Region

  adding: JKT48_Posters_REVISI_Region/ (stored 0%)
  adding: JKT48_Posters_REVISI_Region/Revisi_41_Kathrina.png (deflated 39%)
  adding: JKT48_Posters_REVISI_Region/Revisi_25_Delynn.png (deflated 41%)
  adding: JKT48_Posters_REVISI_Region/Revisi_17_Lana.png (deflated 42%)
  adding: JKT48_Posters_REVISI_Region/Revisi_06_Amanda.png (deflated 34%)
  adding: JKT48_Posters_REVISI_Region/Revisi_45_Kimmy.png (deflated 40%)
  adding: JKT48_Posters_REVISI_Region/Revisi_35_Aralie.png (deflated 40%)
  adding: JKT48_Posters_REVISI_Region/Revisi_27_Olla.png (deflated 41%)
  adding: JKT48_Posters_REVISI_Region/Revisi_36_Oniel.png (deflated 41%)
  adding: JKT48_Posters_REVISI_Region/Revisi_14_Anindya.png (deflated 39%)
  adding: JKT48_Posters_REVISI_Region/Revisi_03_Christy.png (deflated 25%)
  adding: JKT48_Posters_REVISI_Region/Revisi_32_Lyn.png (deflated 41%)
  adding: JKT48_Posters_REVISI_Region/Revisi_44_Ribka.png (deflated 40%)
  adding: JKT48_Posters_REVISI_Region/Revisi_15_Nayla.png (deflated

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
import pandas as pd
import numpy as np
import os

# --- CONFIG OUTPUT ---
output_folder = "JKT48_Final_V3_Fixed"
if not os.path.exists(output_folder): os.makedirs(output_folder)

# Setup Warna
cmap_name = 'GreenToYellow'
cm = mcolors.LinearSegmentedColormap.from_list(cmap_name, ["#00E676", "#FFD600"], N=100)
team_colors = {"Team Love": "#FF69B4", "Team Dream": "#00BFFF", "Team Passion": "#FF4500"}

print(f"🎨 GENERATING POSTER V3 (Layout Rapih + Tanggal Mingguan)...")

if 'full_database' not in locals() or not full_database:
    print("⚠️ Data tidak ditemukan. Jalankan proses mining dulu.")
else:
    # Urutkan
    sorted_members = sorted(full_database.items(), key=lambda x: x[1]['score'], reverse=True)
    count = 0

    for rank, (name, info) in enumerate(sorted_members, 1):
        score, team = info['score'], info['team']

        # --- FILTER: SKIP MEMBER DENGAN SCORE 0 ---
        if score <= 0:
            continue

        count += 1
        daily = np.array(info['daily_trend'])
        t_color = team_colors.get(team, '#FFF')

        # Canvas
        fig = plt.figure(figsize=(10, 12))
        fig.patch.set_facecolor('#121212')
        gs = fig.add_gridspec(3, 2, height_ratios=[0.8, 1.2, 1.5]) # Atur ulang rasio

        # --- A. HEADER ---
        ax_h = fig.add_subplot(gs[0, :]); ax_h.axis('off'); ax_h.set_facecolor('#121212')
        ax_h.text(0.5, 0.7, name.upper(), ha='center', fontsize=50, weight='bold', color='#FFD600')
        ax_h.text(0.5, 0.55, f"  {team.upper()}  ", ha='center', fontsize=15, weight='bold', color='#121212',
                  bbox=dict(facecolor=t_color, edgecolor='none', boxstyle='round,pad=0.4'))
        ax_h.text(0.5, 0.40, f"RANK #{rank}  |  INDEX: {score:.2f}", ha='center', fontsize=14, color='#AAA')

        # Garis Gradasi
        gl = np.linspace(0, 1, 100).reshape(1, -1)
        ax_l = fig.add_axes([0.15, 0.72, 0.7, 0.005]); ax_l.imshow(gl, aspect='auto', cmap=cm); ax_l.axis('off')

        # --- B. GRAFIK HARIAN (DENGAN TANGGAL) ---
        ax_t = fig.add_subplot(gs[1, :]); ax_t.set_facecolor('#121212')
        days = np.arange(len(daily)) # 0 sampai 30

        # Bikin Garis Gradasi
        points = np.array([days, daily]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        lc = LineCollection(segments, cmap=cm, norm=plt.Normalize(daily.min(), daily.max()))
        lc.set_array(daily); lc.set_linewidth(3); ax_t.add_collection(lc); ax_t.autoscale_view()
        ax_t.fill_between(days, daily, color='#00E676', alpha=0.1)

        # Setting Sumbu X (Tanggal Mingguan)
        # Index 0 = Tgl 1, Index 7 = Tgl 8, dst.
        tick_indices = [0, 7, 14, 21, 28]
        tick_labels = ['1 Jan', '8 Jan', '15 Jan', '22 Jan', '29 Jan']

        ax_t.set_xticks(tick_indices)
        ax_t.set_xticklabels(tick_labels, color='white', fontsize=10)
        ax_t.set_title("📈 DAILY TREND (Januari 2026)", fontsize=12, color='white', loc='left', pad=15)

        # Rapikan Frame
        ax_t.spines['top'].set_visible(False)
        ax_t.spines['right'].set_visible(False)
        ax_t.spines['bottom'].set_color('#444')
        ax_t.spines['left'].set_visible(False)
        ax_t.tick_params(axis='y', colors='#666') # Angka Y samar aja
        ax_t.tick_params(axis='x', colors='white')

        # --- C. TOP WILAYAH (STYLE: TEXT DI ATAS BAR) ---
        ax_r = fig.add_subplot(gs[2, 0]); ax_r.set_facecolor('#121212')

        provs = [p.replace("Daerah Istimewa","DI").replace("Kepulauan","Kep.").replace("Sumatera","Sumatra") for p in info['top_regions'].keys()]
        vals = list(info['top_regions'].values())

        # Kita atur posisi Y manual biar renggang
        y_positions = np.arange(len(provs)) * 1.5 # Jarak antar bar 1.5

        # Gambar Bar
        ax_r.barh(y_positions, vals, height=0.6, color=t_color, alpha=0.8)

        # Tulis Nama Wilayah DI ATAS Bar (Biar rapi rata kiri)
        for i, (prov, val) in enumerate(zip(provs, vals)):
            # Nama Wilayah (Putih, di atas bar)
            ax_r.text(0, y_positions[i] + 0.45, prov.upper(), fontsize=9, color='white', weight='bold', ha='left')
            # Nilai (Warna Tim, di ujung kanan bar)
            ax_r.text(val + 2, y_positions[i], str(val), fontsize=9, color=t_color, weight='bold', va='center')

        # Setting Axis Wilayah
        ax_r.set_yticks([]); ax_r.invert_yaxis() # Hilangkan label sumbu Y default
        ax_r.axis('off') # Matikan kotak border total
        ax_r.set_title("🗺️ TOP WILAYAH", fontsize=11, color='white', loc='left', pad=10)

        # --- D. KEYWORDS ---
        ax_k = fig.add_subplot(gs[2, 1]); ax_k.axis('off'); ax_k.set_facecolor('#121212')
        ax_k.set_title("🔍 KEYWORDS", fontsize=11, color='white', loc='left')
        y_txt = 0.95

        kw_data = []
        if info['kw_top'] != ["-"]: kw_data.append(("[TOP SEARCH]", info['kw_top'][:3]))
        if info['kw_rising'] != ["-"]: kw_data.append(("[RISING / VIRAL]", info['kw_rising'][:3]))

        if not kw_data:
            ax_k.text(0, y_txt, "(Tidak ada data spesifik)", fontsize=10, color='#666')

        for label, kws in kw_data:
            ax_k.text(0, y_txt, label, fontsize=10, weight='bold', color='#FFD600')
            y_txt -= 0.1
            for k in kws:
                # Bullet point simpel
                ax_k.text(0, y_txt, f"• {k[:28]}", fontsize=9, color='white')
                y_txt -= 0.08
            y_txt -= 0.05

        plt.tight_layout()
        plt.savefig(f"{output_folder}/Poster_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
        plt.close()

        if count % 5 == 0: print(f"   ... {count} poster tercetak.")

    print(f"\n✅ SELESAI! {count} Poster berhasil dibuat (Member Score 0 di-skip).")

    # Auto ZIP
    print("📦 Sedang membuat ZIP...")
    os.system(f"zip -r JKT48_Posters_V3_Final.zip {output_folder}")
    print("👉 Silakan download file: JKT48_Posters_V3_Final.zip")

🎨 GENERATING POSTER V3 (Layout Rapih + Tanggal Mingguan)...


/tmp/ipython-input-546303689.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Poster_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
/tmp/ipython-input-546303689.py:127: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Poster_{rank:02d}_{name}.

   ... 5 poster tercetak.


/tmp/ipython-input-546303689.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout()
/tmp/ipython-input-546303689.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Poster_{rank:02d}_{name}.png", dpi=300, facecolor

   ... 10 poster tercetak.


/tmp/ipython-input-546303689.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:126: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-546303689.py:127: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Poster_{rank:02d}_{name}.png", dpi=300, facecolor='#121212')
/tmp/ipython-input-546303689.py:127: UserWarning: Glyph 128506 (\N{WORLD MAP}) missing from font(s) DejaVu Sans.
  plt.savefig(f"{output_folder}/Poster_{rank:02d}_{name}.


✅ SELESAI! 12 Poster berhasil dibuat (Member Score 0 di-skip).
📦 Sedang membuat ZIP...
👉 Silakan download file: JKT48_Posters_V3_Final.zip
